# Comprehensive GRB Afterglow Analysis with VegasAfterglow and Redback

This tutorial demonstrates the full capabilities of redback for analyzing GRB afterglows using VegasAfterglow models.

**What you'll learn:**
1. Simulating GRB afterglow data
2. Loading and plotting observational data
3. Maximum likelihood estimation
4. Bayesian parameter estimation
5. Model comparison across different jet structures
6. Handling non-detections and upper limits
7. Advanced features: reverse shocks, jet spreading, SSC

## Installation

```bash
pip install redback VegasAfterglow
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import redback
from redback.constants import day_to_s
import bilby

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Simulating GRB Afterglow Data

Redback's simulation module makes it easy to generate synthetic afterglow data with realistic noise and sampling.

In [ ]:
# Define injection parameters for a power-law jet in ISM
injection_parameters = dict(
    redshift=0.5,
    loge0=52.5,      # log10(Eiso) in erg
    g0=300,          # Initial Lorentz factor  
    thc=0.1,         # Core angle [rad]
    thv=0.3,         # Viewing angle [rad]
    p=2.5,           # Electron index
    logepse=-1.0,    # log10(epsilon_e)
    logepsb=-2.0,    # log10(epsilon_B)
    lognism=0.0,     # log10(n_ISM) [cm^-3]
    loga=-10,        # log10(A*) - effectively no wind
    ke=4.0,          # Energy power-law index
    kg=2.0           # Lorentz factor power-law index
)

# Define observing strategy: X-ray and optical bands
frequencies = [2.4e17, 5e14]  # X-ray (1 keV) and R-band
times = np.logspace(3, 6, 50) * day_to_s  # 1000s to 1e6s

# Create simulated afterglow
afterglow_sim = redback.simulate_transient(
    model='vegas_powerlaw',
    parameters=injection_parameters,
    times=times,
    frequency=frequencies,
    output_format='flux_density',
    noise_term=0.1  # 10% noise
)

print(f"Simulated {len(afterglow_sim.x)} data points")
afterglow_sim.plot_data()

## 2. Loading Real Data

Redback supports multiple data sources:
- Swift BAT+XRT (automatic download)
- Open Access Catalog
- Custom data files

In [ ]:
# Example 1: Load from Swift (requires internet)
# afterglow_real = redback.get_data.get_bat_xrt_afterglow_data_from_swift(grb='GRB070809')

# Example 2: Load from custom CSV files
# Assuming you have data files with columns: time, flux_density, flux_density_err, frequency
# afterglow_real = redback.transient.Afterglow.from_csv(
#     'grb_data.csv',
#     name='GRB_custom',
#     redshift=1.5
# )

# For this tutorial, we'll use the simulated data
afterglow = afterglow_sim

## 3. Maximum Likelihood Estimation

Quick parameter estimation using scipy optimization.

In [ ]:
# Set up priors (will be used as starting points for MLE)
priors = redback.priors.get_priors(model='vegas_powerlaw')

# Run maximum likelihood fit
mle_result = redback.fit_model(
    transient=afterglow,
    model='vegas_powerlaw',
    sampler='scipy',
    priors=priors,
    model_kwargs={'frequency': afterglow.frequency, 'output_format': 'flux_density'},
    outdir='output/mle',
    label='vegas_powerlaw_mle'
)

print("\nMaximum Likelihood Parameters:")
print(mle_result.posterior)

# Plot best fit
mle_result.plot_lightcurve()

## 4. Bayesian Parameter Estimation

Full posterior inference using nested sampling.

In [ ]:
# Run nested sampling (this may take several minutes)
result = redback.fit_model(
    transient=afterglow,
    model='vegas_powerlaw',
    sampler='dynesty',
    nlive=1000,
    priors=priors,
    model_kwargs={'frequency': afterglow.frequency, 'output_format': 'flux_density'},
    outdir='output/bayesian',
    label='vegas_powerlaw_full',
    plot=True,
    resume=True
)

print(f"\nLog Evidence: {result.log_evidence:.2f} ± {result.log_evidence_err:.2f}")

## 5. Posterior Analysis and Visualization

In [ ]:
# Corner plot showing parameter correlations
result.plot_corner(
    parameters=['loge0', 'g0', 'thc', 'thv', 'p', 'logepse', 'logepsb'],
    save=True,
    filename='output/bayesian/corner.png'
)

# Light curve with posterior samples
result.plot_lightcurve(
    random_models=100,
    save=True,
    filename='output/bayesian/lightcurve.png'
)

# Residuals
result.plot_residuals()

## 6. Model Comparison: Different Jet Structures

Compare tophat, Gaussian, and power-law jet structures.

In [ ]:
models_to_compare = ['vegas_tophat', 'vegas_gaussian', 'vegas_powerlaw']
results = {}

for model in models_to_compare:
    print(f"\nFitting {model}...")
    
    # Get appropriate priors for each model
    model_priors = redback.priors.get_priors(model=model)
    
    # Fit model
    results[model] = redback.fit_model(
        transient=afterglow,
        model=model,
        sampler='dynesty',
        nlive=500,
        priors=model_priors,
        model_kwargs={'frequency': afterglow.frequency, 'output_format': 'flux_density'},
        outdir=f'output/comparison/{model}',
        label=model
    )

# Compare evidences
print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
for model, res in results.items():
    print(f"{model:20s}: log(Z) = {res.log_evidence:7.2f} ± {res.log_evidence_err:.2f}")

# Plot all models together
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['blue', 'red', 'green']

for (model, res), color in zip(results.items(), colors):
    # Get maximum likelihood parameters
    max_like_params = res.posterior.iloc[res.posterior['log_likelihood'].idxmax()]
    
    # Plot model
    t_plot = np.logspace(np.log10(afterglow.x.min()), np.log10(afterglow.x.max()), 200)
    # ... (plotting code)
    
plt.legend()
plt.xlabel('Time [s]')
plt.ylabel('Flux Density [mJy]')
plt.xscale('log')
plt.yscale('log')
plt.title('Model Comparison')
plt.savefig('output/comparison/all_models.png', dpi=300)
plt.show()

## 7. Handling Non-Detections

Redback properly handles upper limits in the likelihood.

In [ ]:
# Add some non-detections to the data
# This is done automatically when loading Swift data that includes upper limits

# For custom data, mark non-detections:
afterglow_with_limits = afterglow.copy()
# Set last 10 points as upper limits
afterglow_with_limits.y[-10:] = afterglow.y[-10:] * 3  # 3-sigma upper limit
afterglow_with_limits.y_err[-10:] = np.inf  # Marks as upper limit

# Fit will automatically use appropriate likelihood
result_with_limits = redback.fit_model(
    transient=afterglow_with_limits,
    model='vegas_powerlaw',
    sampler='dynesty',
    nlive=500,
    priors=priors,
    model_kwargs={'frequency': afterglow.frequency, 'output_format': 'flux_density'},
    outdir='output/with_limits',
    label='with_upper_limits'
)

result_with_limits.plot_lightcurve()

## 8. Advanced VegasAfterglow Features

### Reverse Shock

In [ ]:
# Enable reverse shock emission
model_kwargs_rs = {
    'frequency': afterglow.frequency,
    'output_format': 'flux_density',
    'reverse_shock': True,
    'reverse_logepse': -1.5,  # Different microphysics in reverse shock
    'reverse_logepsb': -2.5,
    'reverse_p': 2.3
}

result_rs = redback.fit_model(
    transient=afterglow,
    model='vegas_tophat',
    sampler='dynesty',
    nlive=500,
    priors=redback.priors.get_priors('vegas_tophat'),
    model_kwargs=model_kwargs_rs,
    outdir='output/reverse_shock',
    label='with_reverse_shock'
)

result_rs.plot_lightcurve()

### Jet Spreading

In [ ]:
# Enable jet spreading
model_kwargs_spread = {
    'frequency': afterglow.frequency,
    'output_format': 'flux_density',
    'spreading': True
}

result_spread = redback.fit_model(
    transient=afterglow,
    model='vegas_tophat',
    sampler='dynesty',
    nlive=500,
    priors=redback.priors.get_priors('vegas_tophat'),
    model_kwargs=model_kwargs_spread,
    outdir='output/spreading',
    label='with_spreading'
)

result_spread.plot_lightcurve()

### Synchrotron Self-Compton (SSC)

In [ ]:
# Enable SSC emission (important for high-energy observations)
model_kwargs_ssc = {
    'frequency': afterglow.frequency,
    'output_format': 'flux_density',
    'ssc': True,
    'ssc_cooling': True,
    'kn': True  # Klein-Nishina corrections
}

result_ssc = redback.fit_model(
    transient=afterglow,
    model='vegas_tophat',
    sampler='dynesty',
    nlive=500,
    priors=redback.priors.get_priors('vegas_tophat'),
    model_kwargs=model_kwargs_ssc,
    outdir='output/ssc',
    label='with_ssc'
)

result_ssc.plot_lightcurve()

## 9. Spectral Analysis

Redback can also fit broadband spectra.

In [ ]:
# Simulate multi-frequency observation at fixed time
freq_array = np.logspace(8, 20, 50)  # Radio to X-ray
t_fixed = 1e5  # 1e5 seconds

# Get spectrum
from redback.transient_models.afterglow_models import vegas_tophat

spectrum = vegas_tophat(
    time=np.full_like(freq_array, t_fixed),
    frequency=freq_array,
    output_format='flux_density',
    **injection_parameters
)

# Plot spectrum
plt.figure(figsize=(10, 6))
plt.loglog(freq_array, spectrum * freq_array, 'b-', linewidth=2)
plt.xlabel('Frequency [Hz]', fontsize=14)
plt.ylabel(r'$\nu F_\nu$ [mJy Hz]', fontsize=14)
plt.title(f'Broadband Spectrum at t = {t_fixed/day_to_s:.1f} days', fontsize=14)
plt.grid(alpha=0.3)
plt.show()

## 10. Joint Fitting: Afterglow + Kilonova/Supernova

Redback allows joint fitting of multiple emission components.

In [ ]:
# Example: GRB afterglow + supernova (for low-luminosity GRBs)
# This requires custom model implementation or using redback's combo models

# For GW170817-like events: afterglow + kilonova
# result_joint = redback.fit_model(
#     transient=multi_band_data,
#     model='kilonova_afterglow',  # Combined model
#     sampler='dynesty',
#     nlive=1000,
#     priors=redback.priors.get_priors('kilonova_afterglow'),
#     outdir='output/joint',
#     label='kilonova_plus_afterglow'
# )

print("Joint fitting requires multi-component transient objects - see redback documentation")

## Summary

This tutorial demonstrated:

**Core Features:**
- ✅ Data simulation and loading
- ✅ Quick maximum likelihood fits
- ✅ Full Bayesian parameter estimation
- ✅ Posterior visualization (corner plots, light curves)
- ✅ Model comparison using Bayesian evidence

**Advanced Capabilities:**
- ✅ Non-detection handling (upper limits)
- ✅ Complex likelihood functions
- ✅ VegasAfterglow physics: reverse shocks, spreading, SSC
- ✅ Spectral analysis
- ✅ Multi-component fitting

**Why use redback + VegasAfterglow?**
1. **Speed**: VegasAfterglow provides ~1ms light curves, enabling efficient MCMC
2. **Completeness**: All jet structures (tophat, Gaussian, power-law, step, two-component)
3. **Physics**: Full forward shock + reverse shock, jet dynamics, SSC
4. **Flexibility**: Easy to combine with other redback models (kilonovae, supernovae)
5. **Robustness**: Proper handling of non-detections and complex likelihoods

## Next Steps

- Read the [VegasAfterglow paper](https://ui.adsabs.harvard.edu/abs/2026JHEAp..5000490W/)
- Explore [redback documentation](https://redback.readthedocs.io/)
- Try different jet structures and medium types
- Apply to your own GRB data!

## Available VegasAfterglow Models in Redback

| Model | Jet Structure | Medium | Use Case |
|-------|---------------|---------|----------|
| `vegas_tophat` | Uniform | ISM/Wind/Hybrid | Standard afterglows |
| `vegas_gaussian` | Gaussian | ISM/Wind/Hybrid | Structured jets |
| `vegas_powerlaw` | Power-law | ISM/Wind/Hybrid | Smooth energy profile |
| `vegas_powerlaw_wing` | Power-law + wing | ISM/Wind/Hybrid | Extended wings |
| `vegas_step` | Step function | ISM/Wind/Hybrid | Two-zone jets |
| `vegas_steppowerlaw` | Step + power-law | ISM/Wind/Hybrid | Complex structure |
| `vegas_twocomponent` | Two independent | ISM/Wind/Hybrid | Wide+narrow jets |

All models support:
- Reverse shock emission
- Jet spreading  
- Synchrotron self-Compton
- Magnetar energy injection
- Custom observer angles